In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
import re

sns.set()

In [2]:
response_vars_df1 = pd.read_csv("data/full_unimputed_response_df.csv").iloc[:,1:]
response_vars_df1.shape

(22721, 222)

In [3]:
predictor_vars_df1 = (pd.read_csv("data/full_imputed_predictor_df_iter0.csv")).iloc[:,1:]
#cleaned_cols = [''.join(re.split(r'(\d+)', col)[2:]) for col in predictor_vars_df1.columns]
#predictor_vars_df1.columns = cleaned_cols
predictor_vars_df1.shape

(22721, 157)

In [4]:
def split_cnum(cnum_var):
    cnum_var = cnum_var[1:]
    nums = cnum_var.rstrip('0123456789')
    var_name = cnum_var[len(nums):]
    return var_name

def create_composite_vars(full_df,predictors=True):
    print(full_df.columns)
    if predictors: 
        trs = ""
        colname = "Baseline_Conflict_Score"
        full_df.columns = [col.replace("(","DUMMY").split(")")[0] for col in full_df.columns]
    else:
        trs = "W2"
        colname="Composite_Conflict_Score"
    
    de_var_names = [col for col in full_df.columns if "DE" in col]
    full_df["DE_sum"] = full_df.loc[:,de_var_names].sum(axis=1)

    binary_positive_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl10w2,dncl11w2,dncl12w2,pncl1w2,pncl2w2,"+\
                            "pncl7w2,pncl12w2,pncl13w2,cscaw2,clhcw2,flihcw2,tomenw2,"+\
                            "tomepgw2,tomelnw2,tomesuw2,tomehdw2,dncl9w2,dncl10w2,dncl11w2,"+\
                            "dncl12w2,cbnpw2".replace("w2",trs)).split(",")] if col in full_df.columns]
    binary_negative_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl1w2,dncl2w2,dncl3w2,dncl4w2,dncl5w2,dncl6w2,"+\
                            "dncl7w2,dncl8w2,dncl14w2,pncl3w2,pncl4w2,pncl5w2,pncl6w2,cilw2,cflw2,"+\
                            "cmosw2,cbiw2,cgiw2,tomeew2,tomempw2,tomergw2,"+\
                            "tomerbw2,tomemfw2,tomepmw2,tomethpw2,tomerew2,tomesgw2"\
                                                                     .replace("w2",trs)).split(",")] if col in full_df.columns]
    cat3_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("infdw2,infcw2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,3)] if col in full_df.columns]
    cat4_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn9w2,dn10w2,dn11w2,dn12w2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,4)] if col in full_df.columns]
    cat4_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn14w2,dn1w2,dn2w2,dn3w2,dn4w2,dn5w2,dn6w2,dn7w2,dn8w2".replace("w2",trs))\
                                                 .split(",")]\
                           for i in range(1,4)] if col in full_df.columns]
    cat5_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn1w2,pn2w2,pn10w2,pn11w2,pn12w2,pn13w2".replace("w2",trs)).split(",")]\
                          for i in range(1,5)] if col in full_df.columns]
    cat5_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn3w2,pn4w2,pn5w2,pn6w2,pn7w2,pn9w2".replace("w2",trs)).split(",")]\
                           for i in range(1,5)] if col in full_df.columns]
    #cat3_negative_vars: none
    
    

    full_df["binary_vars"] = full_df.loc[:,binary_positive_vars]\
    .sum(axis=1) - full_df.loc[:,binary_negative_vars].sum(axis=1)

    comp_var = full_df["binary_vars"] - \
    (full_df["DE_sum"] * 0.25)

    return comp_var

# TODO: Add 2< cat variables

In [5]:
predictor_comp_df1 = predictor_vars_df1.copy()
predictor_comp_df1["Baseline_Conflict_Score"] = create_composite_vars(predictor_vars_df1,predictors=True)

response_comp_df1 = response_vars_df1.copy()
response_comp_df1["Composite_Conflict_Score"] = create_composite_vars(response_vars_df1,predictors=False)

trt_comp_df1 = pd.read_csv("data/full_unimputed_trt_df.csv")

complete_vars_df1 = pd.concat([trt_comp_df1,response_vars_df1, response_comp_df1,
                               predictor_vars_df1, predictor_comp_df1],axis=1)

complete_vars_df1.to_csv("data/preprocessed_conflict_data_iter0MICE.csv")

Index(['AGEC_NEW', 'GPA', 'GENDER_(1) Boy', 'GR_(1) 6th grade',
       'GR_(2) 7th grade', 'GR_(3) 8th grade', 'COLL_(1) Yes',
       'COLL_(2) Maybe', 'COLL_(3) Don't know',
       'CELL_(1) Cell phone without Internet',
       ...
       'TOMEMP_(1) Yes', 'TOMELN_(1) Yes', 'TOMERG_(1) Yes', 'TOMERB_(1) Yes',
       'TOMEMF_(1) Yes', 'TOMEPM_(1) Yes', 'TOMETHP_(1) Yes', 'TOMERE_(1) Yes',
       'TOMESG_(1) Yes', 'TOMESU_(1) Yes'],
      dtype='object', length=157)
Index(['DE1', 'DE2', 'DE3', 'DE4', 'DE5', 'DE6', 'DE7', 'DE8', 'DE9', 'DE10',
       ...
       'TOMELNW2_DUMMY1', 'TOMERGW2_DUMMY1', 'TOMERBW2_DUMMY1',
       'TOMEMFW2_DUMMY1', 'TOMEPMW2_DUMMY1', 'TOMETHPW2_DUMMY1',
       'TOMEREW2_DUMMY1', 'TOMESGW2_DUMMY1', 'TOMESUW2_DUMMY1',
       'TOMEHDW2_DUMMY1'],
      dtype='object', length=222)
